# Abstract

Hello fellow kagglers,

The main point of this workbook, when evaluating transition expectation using off policy method, **if we evaluate only using the action of the current agent, we dont know if the reward is due to our choice or the choice of the other agents.**

For example,are we winning because the opposite agent went to a wall or because of our agent did something good ?

So instead of outputting a 4 dimension array , we will evaluate a 4x4 matrix

There s a few genuine method here. The aggresivity method of the different agent , the method of using a .fit on a reinforcment learning (instead of a custom loop using autogradient method) and the Partial Q learning I want to present.

The idea is then instead of using not only the actions of the active agent but also the one of the 4 others.

Therefore , we are not only looking at the expectation of reward knowing our action but the expectation of reward knowing the actions of the 4 agents. 

**Mathematicly, we are estimating the joint probilities of E(state / a1,a2,a3,a4) instead of just E(state,a1) over the average of a2,a3,a4**





We are still using temporal differences to estimate the transition using a dual deep q learning of 4x4 output.
The target network will assume the other agents as well as our will go for the argmax. This time this will be an argmax over 4 actions summed up.
The action network will fit to that target using the 4x4 output mulitplied by the action matrix.

The base of the agent explorer function is from the APEX notebook
The base of the observation_matrix, i have seen in many notebook so would not be able to point its origin.

# The Import

In [ ]:
import numpy as np  # linear algebra
import sys
import os
import tensorflow as tf
import logging
import random
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action
from tensorflow.keras import Model

from kaggle_environments import make
from collections import deque
import tqdm
import pickle
import os as os

from tensorflow.keras.callbacks import Callback

# The agent class and state encoder

In [ ]:
Action_dict, Action_invdict = {}, {}
for index, action in enumerate(Action):
    Action_dict[index] = action.name
    Action_invdict[action.name] = index


def centerize(b):
    dy, dx = np.where(b[:, :, 0])
    centerize_y = (np.arange(0, 7) - 3 + dy[0]) % 7
    centerize_x = (np.arange(0, 11) - 5 + dx[0]) % 11

    b = b[centerize_y, :, :]
    b = b[:, centerize_x, :]

    return b


def observation_matrix(obs_prev, obs, idx):
    b = np.zeros((7 * 11, 17), dtype=np.float32)

    # if len(obs['geese'][idx]) == 0:
    #     return b.reshape(7, 11, -1)

    for p, pos_list in enumerate(obs['geese']):
        # head position
        for pos in pos_list[:1]:
            b[pos, 0 + (p - idx) % 4] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[pos, 4 + (p - idx) % 4] = 1
        # whole position
        for pos in pos_list:
            b[pos, 8 + (p - idx) % 4] = 1

        # previous head position
        for p, pos_list in enumerate(obs_prev['geese']):
            for pos in pos_list[:1]:
                b[pos, 12 + (p - idx) % 4] = 1

                # food
    for pos in obs['food']:
        b[pos, 16] = 1

    b = b.reshape(7, 11, -1)
    b = centerize(b)

    return b

# action encoder

In [ ]:
def action_matrix(actions, current_index):
    action_one_hot = np.zeros((4, 4))
    for idx, act in enumerate(actions):
        if act:
            if idx == current_index:
                action_one_hot[(idx - current_index) % 4, Action_invdict[act]] = 1
            else:
                action_one_hot[(idx - current_index) % 4, Action_invdict[act]] = -1/4

    return action_one_hot


def get_info(agent, prev_state, state, step, idx=0, train=False, epsilon=0.1):
    act = state[idx]["action"]
    if state[idx]["status"] != "ACTIVE":
        return None, None

    matrix = observation_matrix(prev_state[0].observation, state[0].observation, idx)

    act_index = Action_invdict[act]
    if epsilon == 2:
        return matrix, Action_dict[np.random.choice(4)]

    if train and (np.random.random() < epsilon):
        return matrix, Action_dict[((act_index - 2) + (1 + np.random.choice(2))) % 4]

    Q = tf.squeeze(agent.actor([matrix.reshape(1, 7, 11, 17), step.reshape(1, 1) / 200])).numpy()
    return matrix, Action_dict[np.argmax(Q[0, :])]

# reward function

it ouput two values , one for your action and one for the one of your opponent

In [ ]:
def reward(prev_state, state):
    fruit = 0.1#0.2
    rewards  = np.zeros(4)*0.01#surviving is good
    rewards2 = np.zeros(4)*0.0

    prev_guesses = prev_state[0]["observation"]["geese"]
    glen = np.array([len(prev_guesses[idx])for idx in range(4)])
    guesses = state[0]["observation"]["geese"]

    active = np.array([s.status == "ACTIVE" for s in prev_state])
    n_active = sum(active)

    done = all(s["status"] != "ACTIVE" for s in state)

    if not done:
        for idx in range(4):
            if not active[idx]:  # dead
                continue

            if len(guesses[idx]) == 0 and active[idx]:  # died on this turn
                rewards[idx] += -1
                rewards2[:idx] += 1 #/ n_active
                rewards2[idx + 1:] += 1 #/ n_active

            if len(guesses[idx]) - len(prev_guesses[idx]) > 0:  # ate a fruit
                rewards[idx] += fruit
                rewards2[:idx] += -fruit #/ n_active
                rewards2[idx + 1:] += -fruit #/ n_active

    if done:
        f = lambda s: s.reward if s.reward else 0
        env_reward = np.array([f(s) for s in state])
        
        rewards +=  3*(env_reward == max(env_reward))  # 1 to winner
        rewards +=  -0.5*(env_reward != max(env_reward))

    return rewards,rewards2


# agent Class

In [ ]:
def snakeConv():
    i = tf.keras.layers.Input(shape=(7, 11, 17), name="matrix")
    istep = tf.keras.layers.Input(shape=(1), name="step")

    x = tf.keras.layers.Conv2D(200, 2, activation="relu")(i)
    x = tf.keras.layers.AveragePooling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(100, 2, activation="relu")(x)
    x = tf.keras.layers.AveragePooling2D((2,2))(x)

    x = tf.keras.layers.Flatten()(x)
    y = tf.reduce_mean(i, axis=[1, 2])

    z = tf.keras.layers.concatenate([x,y, istep])
    z = tf.keras.layers.Dense(100, activation="relu")(z)
    z = tf.keras.layers.Dense(100, activation="relu")(z)

    z = tf.keras.layers.Dense(4 * 4)(z)
    qvalues = tf.keras.layers.Reshape((4, 4))(z)

    #qvalues = tf.clip_by_value(qvalues, -2, 3)

    actor = Model(inputs=[i, istep], outputs=qvalues)
    actor.compile()

    return actor


def model_compile(actor):
    i = tf.keras.layers.Input(shape=(7, 11, 17), name="matrix")
    istep = tf.keras.layers.Input(shape=(1), name="step")
    act_onehot = tf.keras.layers.Input(shape=(4, 4), name="onehot")

    x = tf.reduce_sum(actor([i, istep]) * act_onehot, axis=[1, 2])

    model_to_fit = Model(inputs=[i, istep, act_onehot], outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    model_to_fit.compile(opt, "MSE")
    return model_to_fit


def model_target(target):
    i = tf.keras.layers.Input(shape=(7, 11, 17), name="matrix")
    istep = tf.keras.layers.Input(shape=(1), name="step")
    i_active_one = tf.keras.layers.Input(shape=(4), name="active_one")
    
    x = target([i, istep])
    
    #constant = tf.constant([1.,-1./4, -1./4, -1./4])
    
    x = tf.reduce_max(x, axis=2)
    logits = tf.reduce_sum(x*i_active_one, axis=1)


    model_to_target = Model(inputs=[i, istep,i_active_one], outputs=logits)
    return model_to_target


class agent_class():
    def __init__(self, alpha):
        self.actor = snakeConv()
        self.target = tf.keras.models.clone_model(self.actor)
        self.model_fit = model_compile(self.actor)
        self.label = model_target(self.target)

        self.alpha = alpha

        self.prevState = None
        self.step = None
        self.opt = tf.keras.optimizers.Adam(0.001, clipnorm=0.25)#0.001

    def agent_call(self, state, step, index):
        if self.prevState == None:
            matrix = observation_matrix(state, state, index)
        else:
            matrix = observation_matrix(self.prevState, state)

        logits = np.array(self.actor(matrix.reshape(1, 7, 11, 17), step.reshape(1, 1) / 200))[0]
        action = np.argmax(logits[index])
        action_name = Action_dict[action]
        self.prevState = state

        return action_name

    def reset(self):
        self.prevState = None

    def update(self):
        self.target.set_weights(self.actor.get_weights())

# Explorer and Fit Method

In [ ]:
def one_path(epsilon, train=True):
    done = False
    random_agent = [agents[random.randint(0, nb_of_agents - 1)] for _ in range(4)]

    state = env.reset(4)
    prev_state = state

    epoch = 0
    while not done:
        # get action
        epoch += 1
        active = np.array([s.status == "ACTIVE" for s in state])
        infos = [get_info(random_agent[idx], prev_state, state, np.array(epoch), idx, train=train, epsilon=epsilon)
                 for idx in range(4)]

        ## STEPS INTO
        actions = [info[1] for info in infos]
        prev_state = state
        state = env.step(actions)
        prev_dones = [s["status"] != "ACTIVE" for s in prev_state]
        dones = [s["status"] != "ACTIVE" for s in state]
        done = all(dones)
        rewards,rewards2 = reward(prev_state, state)

        ## Memories
        if train:
            temp = []
            for idx in range(4):
                
                #active indicator that the opponent is alive after actions
                active_one = np.zeros(4)
                active_one[0] = 1
                for opponent in range(4):
                    if opponent != idx and not dones[opponent]:
                        active_one[(opponent-idx)%4] = -1/4
                
                
                if epoch > 1 and active[idx] and not prev_dones[idx]:
                    if not dones[idx]:
                        temp.append((infos[idx][0],
                                       action_matrix(actions, idx), (rewards[idx],rewards2[idx]),
                                       observation_matrix(prev_state[0].observation, state[0].observation, idx)
                                       , dones[idx], epoch,active_one))
                    else:
                        temp.append((infos[idx][0],
                                       action_matrix(actions, idx), (rewards[idx],rewards2[idx]),
                                       infos[idx][0]
                                       , dones[idx], epoch,active_one))
            memory.append(temp) # add all the transitions in the same tuples
            
    return epoch


def explorer(episodes, epsilon):
    with tqdm.trange(episodes) as t:
        for j in t:
            epoch = one_path(epsilon)
            t.set_description(f'Episode {j}')
            t.set_postfix(len=epoch, memory_start=len(memory))

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='loss', value=0.05, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True


def all_fit(agents_list, epochs=1):
    for agent in agents_list:
        if len(memory) // 4 > batch_size:
            samples_Lists = random.sample(memory, min(max_batch,len(memory)))
            samples = sum(samples_Lists, [])
            
            matrix = tf.cast(tf.stack(np.array([sample[0] for sample in samples])), tf.float32)
            action_one_hot = tf.cast(tf.stack(np.array([sample[1] for sample in samples])), tf.float32)
            
            reward1 = tf.cast(tf.stack(np.array([sample[2][0] for sample in samples])), tf.float32)
            reward2 = tf.cast(tf.stack(np.array([sample[2][1] for sample in samples])), tf.float32)
            reward = reward1 + agent.alpha*reward2
            
            matrix_ = tf.cast(tf.stack(np.array([sample[3] for sample in samples])), tf.float32)
            done = tf.cast(tf.stack(np.array([sample[4] for sample in samples])), tf.float32)
            step = tf.cast(tf.stack(np.array([sample[5] for sample in samples])), tf.float32)/200
            
            active_one = tf.cast(tf.stack(np.array([sample[6] for sample in samples])), tf.float32)

            labels_TD = tf.clip_by_value(agent.label([matrix_, step,active_one]) * (1 - done) + reward, tf_clip_min, tf_clip_max) 

            agent.model_fit.fit([matrix, step, action_one_hot]
                          , labels_TD, epochs=epochs, verbose=1, batch_size=batch_size, shuffle=False)#no shuffle to keep actions in the same gradient


def all_fit_update(agents_list, epochs=1000):
    for agent in agents_list:
        if len(memory) // 4 > batch_size:
            samples_Lists = random.sample(memory, min(max_batch,len(memory)))
            samples = sum(samples_Lists, [])
            
            matrix = tf.cast(tf.stack(np.array([sample[0] for sample in samples])), tf.float32)
            action_one_hot = tf.cast(tf.stack(np.array([sample[1] for sample in samples])), tf.float32)
           
            reward1 = tf.cast(tf.stack(np.array([sample[2][0] for sample in samples])), tf.float32)
            reward2 = tf.cast(tf.stack(np.array([sample[2][1] for sample in samples])), tf.float32)
            reward = reward1 + agent.alpha*reward2
            
            matrix_ = tf.cast(tf.stack(np.array([sample[3] for sample in samples])), tf.float32)
            done = tf.cast(tf.stack(np.array([sample[4] for sample in samples])), tf.float32)
            step = tf.cast(tf.stack(np.array([sample[5] for sample in samples])), tf.float32)/200
            
            active_one = tf.cast(tf.stack(np.array([sample[6] for sample in samples])), tf.float32)

            labels_TD = tf.clip_by_value(agent.label([matrix_, step,active_one]) * (1 - done)+ reward , tf_clip_min, tf_clip_max) 

            agent.model_fit.fit([matrix, step, action_one_hot]
                          , labels_TD, epochs=epochs, verbose=1, batch_size=batch_size, shuffle=False)#no shuffle to keep actions in the same gradient

            agent.update()

            labels_TD = tf.clip_by_value(agent.label([matrix_, step,active_one]) * (1 - done) + reward, tf_clip_min, tf_clip_max) 
            agent.model_fit.fit([matrix, step, action_one_hot]
                                , labels_TD, epochs=epochs, verbose=1, batch_size=batch_size, shuffle=False)#no shuffle to keep actions in the same gradient
            agent.update()

# Main loop

In [ ]:
env = make("hungry_geese", debug=False)

memory = deque(maxlen=100_000)
nb_of_agents = 4
batch_size = 256

convergence = 0.05
max_batch = 20_000//batch_size*batch_size//3 #around 20K transitions
tf_clip_min = -100
tf_clip_max =  100

agents = []
alphas = np.linspace(0,0.5,nb_of_agents)
#alphas = [0.0,0.0,0.0]

for i in range(nb_of_agents):
    agents.append(agent_class(alphas[i]))

In [ ]:
explorer(100, 0.50)
all_fit(agents, 1)
Epoch = 1

for i in range(1_000):
    epsilon = 0.25
    if i > 100: epsilon = 0.10
    if i > 200: epsilon = 0.05
    if i > 500: epsilon = 0.02
    if i > 750: epsilon = 0.01

    explorer(10, epsilon)
    all_fit(agents, Epoch)

    if i % 50 == 0 and i!= 0:
        all_fit_update(agents, Epoch)

            
    if i % 100 == 99:  
        try:
            os.mkdir(str(i))
            for p, agent in enumerate(agents):
                agent.actor.save_weights(str(i) + "/agent.ckp" + str(p))
        except:
            pass
    
    if i % 5 == 0:
        try:
            for p, agent in enumerate(agents):
                agent.actor.save_weights('agent.ckp' + str(p))
        except:
            print("save problem")

    
        runlen = one_path(0, True)
        print("\ ")
        print("episode" + str(i) + "--" + str(runlen))

In [ ]:
%%writefile main.py

import numpy as np  # linear algebra
import tensorflow as tf
from tensorflow.keras import Model
import os as os
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action
import sys

ext_folder = '/kaggle_simulations/agent/'
sys.path.append(ext_folder)
model_name = 'agent.ckp'+ str(1)

Action_dict, Action_invdict = {}, {}
for index, action in enumerate(Action):
    Action_dict[index] = action.name
    Action_invdict[action.name] = index


def centerize(b):
    dy, dx = np.where(b[:, :, 0])
    centerize_y = (np.arange(0, 7) - 3 + dy[0]) % 7
    centerize_x = (np.arange(0, 11) - 5 + dx[0]) % 11

    b = b[centerize_y, :, :]
    b = b[:, centerize_x, :]

    return b


def observation_matrix(obs_prev, obs, idx):
    b = np.zeros((7 * 11, 17), dtype=np.float32)

    # if len(obs['geese'][idx]) == 0:
    #     return b.reshape(7, 11, -1)

    for p, pos_list in enumerate(obs['geese']):
        # head position
        for pos in pos_list[:1]:
            b[pos, 0 + (p - idx) % 4] = 1
        # tip position
        for pos in pos_list[-1:]:
            b[pos, 4 + (p - idx) % 4] = 1
        # whole position
        for pos in pos_list:
            b[pos, 8 + (p - idx) % 4] = 1

        # previous head position
        for p, pos_list in enumerate(obs_prev['geese']):
            for pos in pos_list[:1]:
                b[pos, 12 + (p - idx) % 4] = 1

                # food
    for pos in obs['food']:
        b[pos, 16] = 1

    b = b.reshape(7, 11, -1)
    b = centerize(b)

    return b

############## MODEL


def snakeConv():
    i = tf.keras.layers.Input(shape=(7, 11, 17), name="matrix")
    istep = tf.keras.layers.Input(shape=(1), name="step")

    x = tf.keras.layers.Conv2D(200, 2, activation="relu")(i)
    x = tf.keras.layers.AveragePooling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(100, 2, activation="relu")(x)
    x = tf.keras.layers.AveragePooling2D((2,2))(x)

    x = tf.keras.layers.Flatten()(x)
    y = tf.reduce_mean(i, axis=[1, 2])

    z = tf.keras.layers.concatenate([x,y, istep])
    z = tf.keras.layers.Dense(100, activation="relu")(z)
    z = tf.keras.layers.Dense(100, activation="relu")(z)

    z = tf.keras.layers.Dense(4 * 4)(z)
    qvalues = tf.keras.layers.Reshape((4, 4))(z)

    #qvalues = tf.clip_by_value(qvalues, -2, 3)

    actor = Model(inputs=[i, istep], outputs=qvalues)
    actor.compile()

    return actor

class agent_class():
    def __init__(self, alpha):
        self.actor = snakeConv()
        self.alpha = alpha
        self.prevState = None
        self.step = None


    def agent_call(self,state,index):
        if self.prevState == None:matrix      = observation_matrix(state,state,index)
        else:                     matrix      = observation_matrix(self.prevState,state,index)

        if self.step == None:step=1
        else:self.step += 1
        qvalues        = agent1.actor( [matrix.reshape(1,7,11,17), np.array(step).reshape(1, 1) / 200] )[0]
        self.prevState = state
        return Action_dict[np.argmax(qvalues[0, :])]
                 
agent1 = agent_class(0)                                    
agent1.actor.load_weights(os.path.join(ext_folder, model_name))

def agent(obs_dict, config_dict):
    return agent1.agent_call(obs_dict,obs_dict.index)


In [ ]:
!tar -czvf submission.tar.gz main.py agent.ckp1.*